In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import qubex as qx

from qubex.simulator import (
    Coupling,
    QuantumSimulator,
    QuantumSystem,
    Transmon,
)

In [ ]:
q1 = Transmon(
    label="Q01",
    dimension=3,
    frequency=7.648,
    anharmonicity=-0.33,
    # relaxation_rate=0.00005,
    # dephasing_rate=0.00005,
)

q2 = Transmon(
    label="Q02",
    dimension=3,
    frequency=8.275,
    anharmonicity=-0.33,
    # relaxation_rate=0.00005,
    # dephasing_rate=0.00005,
)

g12 = Coupling(
    pair=(q1.label, q2.label),
    strength=0.0123,
)

system = QuantumSystem(
    objects=[q1, q2],
    couplings=[g12],
)

simulator = QuantumSimulator(system)

In [ ]:
g = 2 * np.pi * g12.strength
Delta = 2 * np.pi * (q1.frequency - q2.frequency)
alpha1 = 2 * np.pi * q1.anharmonicity
alpha2 = 2 * np.pi * q2.anharmonicity

xi = g**2 * (alpha1 + alpha2) / ((Delta + alpha1) * (Delta - alpha2))

print(f"ξ = {xi / (2 * np.pi) * 1e6:.2f} kHz")

In [ ]:
pulse_duration = 30  # ns
ramp_duration = 10  # ns
x90 = qx.pulse.FlatTop(
    duration=pulse_duration,
    amplitude=0.5 * np.pi / (pulse_duration - ramp_duration),
    tau=ramp_duration,
)
x90.plot(divide_by_two_pi=True)

x180 = qx.pulse.FlatTop(
    duration=pulse_duration,
    amplitude=np.pi / (pulse_duration - ramp_duration),
    tau=ramp_duration,
)
x180.plot(divide_by_two_pi=True)

In [ ]:
def jazz_sequence(T: float) -> qx.PulseSchedule:
    with qx.PulseSchedule(
        [
            qx.PulseChannel(
                label=q1.label,
                frequency=q1.frequency,
                target=q1.label,
            ),
            qx.PulseChannel(
                label=q2.label,
                frequency=q2.frequency,
                target=q2.label,
            ),
        ]
    ) as ps:
        ps.add(q1.label, x90)
        ps.add(q1.label, qx.Blank(T // 2))
        ps.barrier()
        ps.add(q1.label, x180)
        ps.add(q2.label, x180)
        ps.add(q1.label, qx.Blank(T // 2))
        ps.add(q1.label, x90.scaled(-1))
    
    return ps

In [ ]:
jazz_t100 = jazz_sequence(100)

jazz_t100.plot(
    title="JAZZ Sequence : T = 100 ns",
)

jazz_t200 = jazz_sequence(200)

jazz_t200.plot(
    title="JAZZ Sequence : T = 200 ns",
)

In [ ]:
result = simulator.mesolve(
    controls=jazz_t100,
    initial_state=system.state(
        {
            q1.label: "0",
            q2.label: "0",
        },
    ),
    n_samples=1001,
)

result.plot_population_dynamics(q1.label)
result.plot_population_dynamics(q2.label)
result.plot_bloch_vectors(q1.label)
result.plot_bloch_vectors(q2.label)
result.display_bloch_sphere(q1.label)
result.display_bloch_sphere(q2.label)

In [ ]:
time_range = np.arange(0, 3001, 200)
results = []
for T in time_range:
    res = simulator.mesolve(
        controls=jazz_sequence(T),
        initial_state=system.state(
            {
                q1.label: "0",
                q2.label: "0",
            },
        ),
        n_samples=2,
    )
    p_1 = res.get_substates(q1.label)[-1].diag()[1]
    results.append(p_1)

In [ ]:
from qubex.analysis.fitting import fit_cosine

fit_result = fit_cosine(
    time_range,
    results,
    xaxis_title="Time (ns)",
    yaxis_title="P(1)",
)

xi_fit = fit_result["frequency"]

print(f"ξ (actual) : {xi / (2 * np.pi) * 1e6:.2f} kHz")
print(f"ξ (fit)    : {xi_fit * 1e6:.2f} kHz")